# Data Cleaning

In [1]:
# Install Lib

In [2]:
import pandas as pd
import numpy as np
import os
from scipy import stats

In [3]:
# File Paths

In [4]:


raw_path = "C:/Users/user/Desktop/main/AI_Smart_City/data"
cleaned_path = "C:/Users/user/Desktop/main/AI_Smart_City/cleaned"

os.makedirs(cleaned_path, exist_ok=True)

In [5]:
# TRAFFIC DATA CLEANING

In [6]:
print("Loading Traffic Dataset...")
traffic = pd.read_csv(os.path.join(raw_path, "C:/Users/user/Desktop/main/AI_Smart_City/data/Metro_Interstate_Traffic_Volume.csv"))

print("Shape before cleaning:", traffic.shape)
print(traffic.head())

Loading Traffic Dataset...
Shape before cleaning: (44651, 9)
  holiday    temp  rain_1h  snow_1h  clouds_all weather_main  \
0     NaN  288.28      0.0      0.0          40       Clouds   
1     NaN  289.36      0.0      0.0          75       Clouds   
2     NaN  289.58      0.0      0.0          90       Clouds   
3     NaN  290.13      0.0      0.0          90       Clouds   
4     NaN  291.14      0.0      0.0          75       Clouds   

  weather_description            date_time  traffic_volume  
0    scattered clouds  2012-10-02 09:00:00            5545  
1       broken clouds  2012-10-02 10:00:00            4516  
2     overcast clouds  2012-10-02 11:00:00            4767  
3     overcast clouds  2012-10-02 12:00:00            5026  
4       broken clouds  2012-10-02 13:00:00            4918  


In [7]:
# Handle missing values 

In [8]:
traffic = traffic.dropna(subset=['date_time', 'traffic_volume'])
traffic = traffic.fillna(traffic.median(numeric_only=True))

In [9]:
# Convert date column 

In [10]:
traffic['date_time'] = pd.to_datetime(traffic['date_time'], errors='coerce')
traffic = traffic.dropna(subset=['date_time'])

In [11]:
# Remove duplicates

In [12]:
traffic = traffic.drop_duplicates()

In [13]:
# --- Remove outliers using IQR ---

In [14]:

num_cols = traffic.select_dtypes(include=np.number).columns

Q1 = traffic[num_cols].quantile(0.25)
Q3 = traffic[num_cols].quantile(0.75)
IQR = Q3 - Q1


traffic = traffic[~((traffic[num_cols] < (Q1 - 1.5 * IQR)) | (traffic[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

print("Outliers removed using IQR method")
print(" Shape after cleaning:", traffic.shape)


Outliers removed using IQR method
 Shape after cleaning: (44651, 9)


In [15]:
# Save cleaned dataset 

In [16]:

output_path = os.path.join(cleaned_path, "traffic_cleaned.csv")
traffic.to_csv(output_path, index=False)
print(" Saved  traffic_cleaned.csv")


 Saved  traffic_cleaned.csv


In [17]:
# AIR QUALITY DATA CLEANING

In [18]:
print(" Loading Air Quality Dataset...")
aqi = pd.read_csv(os.path.join(raw_path, "AirQualityUCI.csv"))

print(" Shape before cleaning:", aqi.shape)
print(aqi.head())

 Loading Air Quality Dataset...
 Shape before cleaning: (9471, 17)
        Date      Time  CO(GT)  PT08.S1(CO)  NMHC(GT)  C6H6(GT)  \
0  3/10/2004  18:00:00     2.6       1360.0     150.0      11.9   
1  3/10/2004  19:00:00     2.0       1292.0     112.0       9.4   
2  3/10/2004  20:00:00     2.2       1402.0      88.0       9.0   
3  3/10/2004  21:00:00     2.2       1376.0      80.0       9.2   
4  3/10/2004  22:00:00     1.6       1272.0      51.0       6.5   

   PT08.S2(NMHC)  NOx(GT)  PT08.S3(NOx)  NO2(GT)  PT08.S4(NO2)  PT08.S5(O3)  \
0         1046.0    166.0        1056.0    113.0        1692.0       1268.0   
1          955.0    103.0        1174.0     92.0        1559.0        972.0   
2          939.0    131.0        1140.0    114.0        1555.0       1074.0   
3          948.0    172.0        1092.0    122.0        1584.0       1203.0   
4          836.0    131.0        1205.0    116.0        1490.0       1110.0   

      T    RH      AH  Unnamed: 15  Unnamed: 16  
0  13

In [19]:
# Handle missing values

In [20]:
aqi = aqi.fillna(aqi.median(numeric_only=True))


In [21]:
# Remove duplicates

In [22]:
aqi = aqi.drop_duplicates()

In [23]:

# Remove outliers using IQR 

In [24]:
#  Remove outliers using IQR (only for numeric columns)

In [25]:

num_cols = aqi.select_dtypes(include=['number']).columns

Q1 = aqi[num_cols].quantile(0.25)
Q3 = aqi[num_cols].quantile(0.75)
IQR = Q3 - Q1


aqi = aqi[~((aqi[num_cols] < (Q1 - 1.5 * IQR)) | (aqi[num_cols] > (Q3 + 1.5 * IQR))).any(axis=1)]

print(" Outliers removed using IQR method")
print(" Shape after cleaning:", aqi.shape)


 Outliers removed using IQR method
 Shape after cleaning: (5463, 17)


In [26]:
print(" Shape after cleaning:", aqi.shape)

 Shape after cleaning: (5463, 17)


In [27]:
# Save cleaned dataset

In [28]:
aqi.to_csv(os.path.join(cleaned_path, "air_quality_cleaned.csv"), index=False)
print(" Saved → air_quality_cleaned.csv\n")

 Saved → air_quality_cleaned.csv



In [29]:
# ENERGY CONSUMPTION DATA CLEANING

In [30]:


print(" Loading Energy Dataset...")

energy_file = os.path.join(raw_path, "household_power_consumption.txt")

if not os.path.exists(energy_file):
    print(" File not found:", energy_file)
else:
    energy = pd.read_csv(energy_file, sep=';', low_memory=False)
    
    print(" Shape before cleaning:", energy.shape)
    print(energy.head())


 Loading Energy Dataset...
 Shape before cleaning: (2075259, 9)
         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  


In [31]:
print(" Shape before cleaning:", energy.shape)
print(energy.head())


 Shape before cleaning: (2075259, 9)
         Date      Time Global_active_power Global_reactive_power  Voltage  \
0  16/12/2006  17:24:00               4.216                 0.418  234.840   
1  16/12/2006  17:25:00               5.360                 0.436  233.630   
2  16/12/2006  17:26:00               5.374                 0.498  233.290   
3  16/12/2006  17:27:00               5.388                 0.502  233.740   
4  16/12/2006  17:28:00               3.666                 0.528  235.680   

  Global_intensity Sub_metering_1 Sub_metering_2  Sub_metering_3  
0           18.400          0.000          1.000            17.0  
1           23.000          0.000          1.000            16.0  
2           23.000          0.000          2.000            17.0  
3           23.000          0.000          1.000            17.0  
4           15.800          0.000          1.000            17.0  


In [32]:
# Handle missing values

In [33]:
energy = energy.fillna(method='ffill').fillna(method='bfill')

C:\Users\user\AppData\Local\Temp\ipykernel_10032\1265377183.py:1: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  energy = energy.fillna(method='ffill').fillna(method='bfill')


In [34]:
#  Convert date column


In [35]:
date_cols = [col for col in energy.columns if 'date' in col.lower() or 'time' in col.lower()]
if date_cols:
    energy[date_cols[0]] = pd.to_datetime(energy[date_cols[0]], errors='coerce')
    energy = energy.dropna(subset=[date_cols[0]])

C:\Users\user\AppData\Local\Temp\ipykernel_10032\2605129680.py:3: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  energy[date_cols[0]] = pd.to_datetime(energy[date_cols[0]], errors='coerce')


In [36]:
date_cols = [col for col in energy.columns if 'date' in col.lower() or 'time' in col.lower()]
if date_cols:
    energy[date_cols[0]] = pd.to_datetime(energy[date_cols[0]], errors='coerce')
    energy = energy.dropna(subset=[date_cols[0]])


In [37]:

energy = energy.drop_duplicates()


In [38]:
#  Remove outliers 

In [39]:
num_cols = energy.select_dtypes(include=np.number).columns
energy = energy[(np.abs(stats.zscore(energy[num_cols])) < 3).all(axis=1)]

In [40]:
print("Shape after cleaning:", energy.shape)

Shape after cleaning: (2075259, 9)


In [41]:
energy.to_csv(os.path.join(cleaned_path, "energy_cleaned.csv"), index=False)
print(" Saved  energy_cleaned.csv\n")

 Saved  energy_cleaned.csv



In [42]:
# summary

In [43]:
print("All datasets cleaned successfully!")
print("Files saved in:", cleaned_path)
print(os.listdir(cleaned_path))

All datasets cleaned successfully!
Files saved in: C:/Users/user/Desktop/main/AI_Smart_City/cleaned
['air_quality_cleaned.csv', 'energy_cleaned.csv', 'traffic_cleaned.csv']
